In [ ]:
import ase, ase.io
from mace.calculators import MACECalculator
calculator = MACECalculator(
            model_path="mace.model",
            device="cuda",
            default_dtype="float32",
        )

In [ ]:
atoms = ase.io.read('rollout_0/gentraj_1.xyz', format='xyz', index=":")[1]
atoms_ref = ase.io.read('rollout_0/reftraj_1.xyz', format='xyz', index=":")[1]



In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
mask = np.where(np.array(atoms_ref.get_positions()).ravel()- np.array(atoms.get_positions()).ravel() != 0)
print(mask)

In [ ]:
plt.figure(figsize=(3.5, 3.5))
mask = np.where(np.array(atoms_ref.get_positions()).ravel()- np.array(atoms.get_positions()).ravel() != 0)
plt.scatter(np.array(atoms_ref.get_positions()).ravel(), np.array(atoms.get_positions()).ravel())
plt.plot(plt.xlim(), plt.xlim(), 'k--')
plt.xlabel('Reference positions')
plt.ylabel('Generated positions')


In [ ]:

all_positions = None
all_positions_ref = None
all_energy_atoms = []
all_energy_atoms_ref = []
mean_distances = []
for i_dir in range(0, 30):
    dirname = f'rollout_{i_dir}'
    atoms = ase.io.read(f'{dirname}/gentraj_1.xyz', format='xyz', index=":")[1]
    atoms.set_cell(np.eye(3,3)*25)
    atoms.calc = calculator
    energy_atoms = atoms.get_potential_energy()
    all_energy_atoms.append(energy_atoms)
    atoms_ref = ase.io.read(f'{dirname}/reftraj_1.xyz', format='xyz', index=":")[1]
    atoms_ref.set_cell(np.eye(3,3)*25)
    atoms_ref.calc = calculator
    energy_atoms_ref = atoms_ref.get_potential_energy()
    all_energy_atoms_ref.append(energy_atoms_ref)
    distances = np.linalg.norm(atoms.positions - atoms_ref.positions, axis=-1)
    mean_distances.append(np.mean(distances))
    if all_positions is None:
        all_positions = list(atoms.get_positions())
        all_positions_ref = list(atoms_ref.get_positions())
    all_positions += list(atoms.get_positions())
    all_positions_ref += list(atoms_ref.get_positions())
all_positions = np.array(all_positions)
all_positions_ref = np.array(all_positions_ref)

In [ ]:
plt.figure(figsize=(3.5, 3.5))
plt.scatter(all_positions_ref.ravel(), all_positions.ravel())
plt.plot(plt.xlim(), plt.xlim(), 'k--')
plt.xlabel('Reference positions')
plt.ylabel('Generated positions')

In [ ]:
plt.figure(figsize=(3.5, 3.5))
plt.scatter(all_energy_atoms_ref, all_energy_atoms)
plt.plot(plt.xlim(), plt.xlim(), 'k--')
plt.xlabel('Reference energies')
plt.ylabel('Generated energies')

In [ ]:
# mask_outlier = np.where((np.abs(all_positions.ravel()- all_positions_ref.ravel()) < 2) & (np.abs(all_positions.ravel()- all_positions_ref.ravel()) > 0))
print("MEDIAN ERROR: ", np.median(np.abs(all_positions.ravel()- all_positions_ref.ravel())))
print("MEAN ERROR: ", np.mean(np.abs(all_positions.ravel()- all_positions_ref.ravel())))
print("MAX ERROR: ", np.max(np.abs(all_positions.ravel()- all_positions_ref.ravel())))


In [ ]:
eV_2_kcalmol = 23.0605
print("Median Energy Difference: ", (np.median((np.array(all_energy_atoms) - np.array(all_energy_atoms_ref))))*eV_2_kcalmol)
print("Max Energy Difference: ", (np.array(all_energy_atoms) - np.array(all_energy_atoms_ref)).max()*eV_2_kcalmol)
print("Mean Energy Difference: ", (np.array(all_energy_atoms) - np.array(all_energy_atoms_ref)).mean()*eV_2_kcalmol)
# print((np.array(all_energy_atoms) - np.array(all_energy_atoms_ref)))

In [ ]:
print(np.where(np.array(all_energy_atoms) - np.array(all_energy_atoms_ref) <= 1.58/eV_2_kcalmol)[0].shape[0])
print(np.array(all_energy_atoms).shape[0])
print(np.where(np.array(all_energy_atoms) - np.array(all_energy_atoms_ref) <= 1.58/eV_2_kcalmol)[0].shape[0]/np.array(all_energy_atoms).shape[0])

In [ ]:
select_idx = np.where(np.array(all_energy_atoms) - np.array(all_energy_atoms_ref) <= 1.58/eV_2_kcalmol)[0]
print("Selected indices: ", select_idx)

In [ ]:
dirname = f'rollout_{4}'
atoms = ase.io.read(f'{dirname}/gentraj_1.xyz', format='xyz', index=":")[0]
atoms.set_cell(np.eye(3,3)*25)
atoms.calc = calculator
r_energy_atoms = atoms.get_potential_energy()
print(r_energy_atoms*eV_2_kcalmol)

In [ ]:
print(all_energy_atoms[4]*eV_2_kcalmol-r_energy_atoms*eV_2_kcalmol, all_energy_atoms_ref[4]*eV_2_kcalmol-r_energy_atoms*eV_2_kcalmol)
print(np.array(mean_distances)[4], np.array(mean_distances)[4])

In [ ]:
dirname = f'rollout_{4}'
atoms = ase.io.read(f'{dirname}/gentraj_1.xyz', format='xyz', index=":")[2]
atoms.set_cell(np.eye(3,3)*25)
atoms.calc = calculator
energy_atoms = atoms.get_potential_energy()
print(energy_atoms*eV_2_kcalmol-r_energy_atoms*eV_2_kcalmol)

In [ ]:
eV_2_kcalmol = 23.0605
plt.figure(figsize=(3.5, 3.5))
plt.scatter(np.array(mean_distances)[select_idx], (np.array(all_energy_atoms) - np.array(all_energy_atoms_ref))[select_idx]*eV_2_kcalmol)
plt.axhline(0, c='k', ls='--')
# plt.ylim((np.array(all_energy_atoms) - np.array(all_energy_atoms_ref)).min(), 0.1)
plt.xlabel('r.m.s.d $[\AA]$')
plt.ylabel('$\Delta E\ [kcal/mol]$')

In [ ]:
# mask_outlier = np.where((np.array(all_energy_atoms) - np.array(all_energy_atoms_ref)) < 0.1)

plt.figure(figsize=(3.5,1.5))
_ = plt.hist((np.array(all_energy_atoms) - np.array(all_energy_atoms_ref))[select_idx]*eV_2_kcalmol, bins=100, color='teal')
plt.semilogx()
plt.axvline(1.58, ls="--", c="grey")
plt.xlabel("$\Delta E_{TS}\ (kcal/mol)$")
plt.ylabel("Count")

In [ ]:
3.5/2

In [ ]:

plt.figure(figsize=(3.5,1.5))
_ = plt.hist(mean_distances, bins=1000, color='teal')
plt.semilogx()
plt.xlabel("RMSD $(\AA)$")
plt.ylabel("Count")

In [ ]:
mask_ts = np.where(np.array(mean_distances) > 0.05)[0]

import os
from ase.optimize import BFGS

if os.path.exists("relax.extxyz"): os.remove("relax.extxyz")
for i_dir in mask_ts:
    dirname = f'rollout_{i_dir}'
    traj_ref = ase.io.read(f'{dirname}/reftraj_1.xyz', format='xyz', index=":")
    for i_atoms in range(3):
        atoms_ref = traj_ref[i_atoms]
        atoms_ref.set_cell(np.eye(3,3)*25)
        center = np.mean(atoms_ref.positions, axis=0)
        atoms_ref.positions += 12.5 - center
        if i_atoms in [0,2]:
            atoms_ref.calc = calculator
            # print(energy_atoms)
            opt = BFGS(atoms, logfile='mace_ase.log')
            opt.run(fmax=0.01)
            energy_atoms = atoms_ref.get_potential_energy()
            atoms_ref.info['energy'] = energy_atoms
            atoms_ref.calc = None
            natoms = len(atoms_ref)
            traj_ref[i_atoms].set_array('energy', np.full(natoms, energy_atoms))
    traj_ref[1].info['energy'] = all_energy_atoms_ref[i_dir]
    natoms = len(traj_ref[1])
    traj_ref[1].set_array('energy', np.full(natoms, energy_atoms))
    print("Barrier = ", traj_ref[1].info['energy']-traj_ref[0].info['energy'])
    ase.io.write("relax.extxyz", traj_ref, append=True, format="extxyz")

In [ ]:
from ase import Atoms
from ase.io import read
dirname="rollout_1"
traj_ref = ase.io.read(f'{dirname}/reftraj_1.xyz', format='xyz', index=":")
mol = traj_ref[0]      # ASE Atoms object
# suppose you know atoms 0–1–2 form the angle you want:
theta = mol.get_angle(1, 3, 4, mic=False)
print(f"Angle = {theta:.2f}°")

In [ ]:
from ase import Atoms
from ase.io import read
dirname="rollout_2"
traj_ref = ase.io.read(f'{dirname}/reftraj_1.xyz', format='xyz', index=":")
mol = traj_ref[0]      # ASE Atoms object
# suppose you know atoms 0–1–2 form the angle you want:
theta = mol.get_angle(1, 3, 4, mic=False)
print(f"Angle = {theta:.2f}°")

In [ ]:
from ase import Atoms
from ase.io import read
dirname="rollout_1"
traj = ase.io.read(f'{dirname}/gentraj_1.xyz', format='xyz', index=":")
mol = traj[0]      # ASE Atoms object
# suppose you know atoms 0–1–2 form the angle you want:
theta = mol.get_angle(1, 3, 4, mic=False)
print(f"Angle = {theta:.2f}°")